In [57]:
import pandas as pd
import numpy as np

import _pickle as pkl

# train_df=pd.read_csv('./complaints_train_data_clean.csv',usecols=["category_name","complaint_title","complaint_description",],na_filter=False)

from gensim.parsing.preprocessing import STOPWORDS
def remove_stopwords(tokens):
    # input and outputs a list of words
    return [word for word in tokens if word not in STOPWORDS]

train_df=pd.read_csv('./complaints_train_validation_data_clean.csv',usecols=["index","category_name","complaint_title","complaint_description",],na_filter=False)

whitelist = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ')

sentences = (train_df['complaint_title']+" "+train_df['complaint_description']).tolist()
sentences = [''.join(filter(whitelist.__contains__, x)).lower() for x in sentences]
sentences = [x.split() for x in sentences]
sentences = list(map(remove_stopwords, sentences))
train_df.insert(1,"complaint_text",sentences)


val_df=pd.read_csv('./complaints_validation_data_clean.csv',usecols=["category_name","complaint_title","complaint_description",],na_filter=False)

test_df=pd.read_csv('./complaints_test_data_clean.csv',usecols=["category_name","complaint_title","complaint_description",],na_filter=False)

## gold labels
gold_df=pd.read_csv('./clean-gold-labels.tsv',sep='\t',usecols=["category_name","complaint_description",],na_filter=False)
gold_df['complaint_title'] = gold_df['complaint_description']
sentences = (gold_df['complaint_description']).tolist()
sentences = [''.join(filter(whitelist.__contains__, x)).lower() for x in sentences]
sentences = [x.split() for x in sentences]
sentences = list(map(remove_stopwords, sentences))
gold_df.insert(1,"complaint_text",sentences)
######

sentences = (test_df['complaint_title']+" "+test_df['complaint_description']).tolist()
sentences = [''.join(filter(whitelist.__contains__, x)).lower() for x in sentences]
sentences = [x.split() for x in sentences]
sentences = list(map(remove_stopwords, sentences))
test_df.insert(1,"complaint_text",sentences)

train_df.head()

,index,complaint_text,category_name,complaint_title,complaint_description
0,39735,"[complaint, garbage, issue, huge, garbage, isl...",Garbage,Complaint About Garbage Issue,A huge garbage island close to martin luther p...
1,20978,"[complaint, bmtc, hi, bus, service, silver, co...",New Bus Shelters,Complaint Against BMTC,Hi.. There is no bus service in silver county ...
2,11133,"[garbage, dumps, main, road, lots, garbage, du...",Garbage,Garbage dumps in main road,Lots of garbage dumping in the main road and l...
3,44549,"[road, condition, ejipura, sewage, pipe, line,...",Bad Roads,Road Condition of Ejipura after the sewage pip...,"Dear All,\n\nI am writing this email to you as..."
4,5108,"[street, lights, time, near, bagmane, tech, pa...",Repair of streetlights,Street Lights are Off almost all the time near...,Street Lights are Off almost all the time near...


In [58]:
colsize = len(train_df['category_name'])
train_df['category_name'] = train_df["category_name"].astype('category')
#train_df['true_label'] = pd.Series(np.zeros(colsize), index=train_df.index)
# train_df['predicted_label'] = pd.Series(np.zeros(colsize), index=train_df.index)

train_df['true_label'] = train_df['category_name'].cat.codes
# for i in range(colsize):
#     if(train_df['true_label'][i]==5):
#         print(train_df['true_label'][i],train_df['category_name'][i])

In [59]:
for i,x in enumerate(train_df.groupby("category_name").agg({"complaint_title": np.count_nonzero}).index._data):
    print(i,x)

0 Air Pollution
1 Autorickshaws and Taxis
2 BMTC - Driver or Conductor
3 BMTC - Need new Bus Route
4 BMTC - Others
5 Bad Roads
6 Broken Storm Water Drains
7 Cattle
8 Clearing of Blockage of Under Ground Drainage Pipelines and Replacement of Damaged or Missing Manhole Cover
9 Desilting - Lakes
10 Diseases
11 Electricity
12 Flooding of Roads and Footpaths
13 Footpaths
14 Garbage
15 Government Land Encroachment
16 Hawkers and Vendors
17 Hoardings
18 Illegal posters and Hoardings
19 Lakes - Others
20 Maintenance of Roads and Footpaths - Others
21 Manholes
22 Mosquitos
23 Need New Streetlights
24 Need New Toilets 
25 New Bus Shelters
26 No Sewage Drains
27 Noise Pollution
28 Others
29 Overflow of Storm Water Drains
30 Parking Violations
31 Parks and playgrounds
32 Potholes
33 Public Nuisance
34 Repair of streetlights
35 Sewage and Storm Water Drains - Others
36 Stray Dogs
37 Traffic
38 Trees, Parks and Playgrounds - Others
39 Unauthorized Construction
40 Water Leakage
41 Water Supply 


In [60]:
class_names = train_df.groupby("category_name").agg({"complaint_title": np.count_nonzero}).index.tolist()

pkl.dump(class_names,open("class_names.p","wb"))

noOfClasses = len(train_df.groupby("category_name").agg({"complaint_title": np.count_nonzero}).index)

print(class_names)


['Air Pollution', 'Autorickshaws and Taxis', 'BMTC - Driver or Conductor', 'BMTC - Need new Bus Route', 'BMTC - Others', 'Bad Roads', 'Broken Storm Water Drains', 'Cattle', 'Clearing of Blockage of Under Ground Drainage Pipelines and Replacement of Damaged or Missing Manhole Cover', 'Desilting - Lakes', 'Diseases', 'Electricity', 'Flooding of Roads and Footpaths', 'Footpaths', 'Garbage', 'Government Land Encroachment', 'Hawkers and Vendors', 'Hoardings', 'Illegal posters and Hoardings', 'Lakes - Others', 'Maintenance of Roads and Footpaths - Others', 'Manholes', 'Mosquitos', 'Need New Streetlights', 'Need New Toilets ', 'New Bus Shelters', 'No Sewage Drains', 'Noise Pollution', 'Others', 'Overflow of Storm Water Drains', 'Parking Violations', 'Parks and playgrounds', 'Potholes', 'Public Nuisance', 'Repair of streetlights', 'Sewage and Storm Water Drains - Others', 'Stray Dogs', 'Traffic', 'Trees, Parks and Playgrounds - Others', 'Unauthorized Construction', 'Water Leakage', 'Water Supp

In [61]:
keywords = {'Air Pollution':['dust|smoke|burn'],
 'Autorickshaws and Taxis':['taxi','auto','autorickshaw'],
 'BMTC - Driver or Conductor':['rude','behaviour'],
 'BMTC - Need new Bus Route':['frequency'],
 'BMTC - Others':[],
 'Bad Roads':['road'],
 'Broken Storm Water Drains':['overflow','drainage'],
 'Cattle':['cows','buffaloes','goats','cow'],
 'Clearing of Blockage of Under Ground Drainage Pipelines and Replacement of Damaged or Missing Manhole Cover':[],
 'Desilting - Lakes':['lake'],
 'Diseases':['malaria','dengue','cholera','fever','disease','hospital','epidemic'],
 'Electricity':['power','current','power cut'],
 'Flooding of Roads and Footpaths':['water','flood','floods'],
 'Footpaths':['footpath'],
 'Garbage':['waste','plastic','dirt'],
 'Government Land Encroachment':['occupy','illegal'],
 'Hawkers and Vendors':[],
 'Hoardings':['advertise'],
 'Illegal posters and Hoardings':['banner', 'ads ','advertise'],
 'Lakes - Others':['lake'],
 'Maintenance of Roads and Footpaths - Others':[],
 'Manholes':['manhole','man hole'],
 'Mosquitos':['mosquito','mosquitoe','mosquitoes','dengue','malaria'],
 'Need New Streetlights':['streetlight','light','new streetlight'],
 'Need New Toilets ':['toilet','urinal','urinate'],
 'New Bus Shelters':['shelter'],
 'No Sewage Drains':['drainage'],
 'Noise Pollution':['siren','speakers','speakers','loud'],
 'Others':[],
 'Overflow of Storm Water Drains':['pipes'],
 'Parking Violations':['parked','parker'],
 'Parks and playgrounds':['park','play','playground'],
 'Potholes':['holes','pothole'],
 'Public Nuisance':[],
 'Repair of streetlights':['streetlight','light','broken','damaged'],
 'Sewage and Storm Water Drains - Others':['drainage'],
 'Stray Dogs':['dog'],
 'Traffic':['vehicles'],
 'Trees, Parks and Playgrounds - Others':['tree'],
 'Unauthorized Construction':['encroach','building','built'],
 'Water Leakage':[],
 'Water Supply ':[]}

In [62]:
regexPatterns = {'Air Pollution':['air.*pollution|pollution|dust'],
 'Autorickshaws and Taxis':['autorickshaws|taxis|taxi|auto|autorickshaw'],
 'BMTC - Driver or Conductor':['bmtc.*driver|bmtc.*conductor|bus.*driver|bus.*conductor'],
 'BMTC - Need new Bus Route':['bus.*route'],
 'BMTC - Others':['bmtc'],
 'Bad Roads':['bad.*road|road.*bad'],
 'Broken Storm Water Drains':['(broken|damage).*(drain)'],
 'Cattle':['(cattle|cows|buffaloes|goats)'],
 'Clearing of Blockage of Under Ground Drainage Pipelines and Replacement of Damaged or Missing Manhole Cover':['clearing|blockage|under|ground|drainage|pipelines|replacement|damaged|missing|manhole|cover'],
 'Desilting - Lakes':['lake'],
 'Diseases':['diseases|malaria|dengue|cholera'],
 'Electricity':['electricity|power|current|power.*cut'],
 'Flooding of Roads and Footpaths':['((water|flood|flow).*(roads|footpaths))|((roads|footpaths).*(water|flood|flow))'],
 'Footpaths':['footpath'],
 'Garbage':['garbage|waste|plastic|dirt'],
 'Government Land Encroachment':['(government.*land).*(encroach|occupy|illegal)'],
 'Hawkers and Vendors':['(hawkers|vendors)'],
 'Hoardings':['(hoardings|advertisements)'],
 'Illegal posters and Hoardings':['posters|hoardings|banner|ads|advertise'],
 'Lakes - Others':['lake'],
 'Maintenance of Roads and Footpaths - Others':['(maintenance).*(roads|footpaths)'],
 'Manholes':['(manholes|manhole|man hole)'],
 'Mosquitos':['mosquito|mosquitoe|mosquitoes|dengue|malaria'],
 'Need New Streetlights':['(need|no|new).*(streetlight|light)'],
 'Need New Toilets ':['toilets|toilet|urinal|urinate'],
 'New Bus Shelters':['bus.*shelter|shelter.*bus'],
 'No Sewage Drains':['drain'],
 'Noise Pollution':['noise|noise.*pollution|siren|speakers|speakers|loud'],
 'Others':['others'],
 'Overflow of Storm Water Drains':['overflow.*(drains|pipes)'],
 'Parking Violations':['parking|parked|parker'],
 'Parks and playgrounds':['(parks|playgrounds|park|play|playground)'],
 'Potholes':['(pot hole|holes|pothole)'],
 'Public Nuisance':['(public.*nuisance|nuisance)'],
 'Repair of streetlights':['((light).*(repair|broke|damage))|((repair|broke|damage).*(light))'],
 'Sewage and Storm Water Drains - Others':['(sewage|storm|water|drains|drainage)'],
 'Stray Dogs':['(stray|dogs|dog)'],
 'Traffic':['(traffic|vehicles)'],
 'Trees, Parks and Playgrounds - Others':['(trees|parks|playgrounds|tree)'],
 'Unauthorized Construction':['encroach','building','built'],
 'Water Leakage':['water.*leak|leak.*water'],
 'Water Supply ':['water.*supply|supply.*water']}

In [63]:
extracts_df=pd.read_csv('./p.tsv',sep='\t',usecols=["category_name","Entity","complaint words",],na_filter=False)

extracts_df = extracts_df[extracts_df['category_name'].isin(class_names)]
# extracts_df
def combine(x):
    x = x.tolist()
    x = set(x)
    x = '|'.join(list(x)).lower()
    return x
extracts_df = extracts_df.groupby("category_name").agg({"Entity": combine,"complaint words":combine })
extracts_df.to_csv("extracts.csv")
extracts_df


,complaint words,Entity
category_name,,
Air Pollution,wood or garbage burnt|lot of dust due to speed...,garbage|bus
BMTC - Driver or Conductor,rude behavious,gbus
BMTC - Need new Bus Route,increase frequency of buses,bus
BMTC - Others,"extra buses,no proper timing,|no buses|buses s...",bus|traffic
Bad Roads,road bad to commute|roads are not asphalted|ro...,roads|pothole|roads|sewage|footpath|permit
Broken Storm Water Drains,storm water drain filled with garbage|no drain...,drain
Clearing of Blockage of Under Ground Drainage Pipelines and Replacement of Damaged or Missing Manhole Cover,sewage pipies narrow|sewage drain blocked|conn...,sewage|tank
Desilting - Lakes,clean lajke,lake
Diseases,garbage dump|garbage|dead cat at residential p...,garbage|cat


In [64]:
import re
class_words = [ re.sub('-','',x).lower().split() + keywords[x] for x in class_names ]
print(class_words,len(class_words))

[['air', 'pollution', 'dust|smoke|burn'], ['autorickshaws', 'and', 'taxis', 'taxi', 'auto', 'autorickshaw'], ['bmtc', 'driver', 'or', 'conductor', 'rude', 'behaviour'], ['bmtc', 'need', 'new', 'bus', 'route', 'frequency'], ['bmtc', 'others'], ['bad', 'roads', 'road'], ['broken', 'storm', 'water', 'drains', 'overflow', 'drainage'], ['cattle', 'cows', 'buffaloes', 'goats', 'cow'], ['clearing', 'of', 'blockage', 'of', 'under', 'ground', 'drainage', 'pipelines', 'and', 'replacement', 'of', 'damaged', 'or', 'missing', 'manhole', 'cover'], ['desilting', 'lakes', 'lake'], ['diseases', 'malaria', 'dengue', 'cholera', 'fever', 'disease', 'hospital', 'epidemic'], ['electricity', 'power', 'current', 'power cut'], ['flooding', 'of', 'roads', 'and', 'footpaths', 'water', 'flood', 'floods'], ['footpaths', 'footpath'], ['garbage', 'waste', 'plastic', 'dirt'], ['government', 'land', 'encroachment', 'occupy', 'illegal'], ['hawkers', 'and', 'vendors'], ['hoardings', 'advertise'], ['illegal', 'posters', 

In [19]:
########### discrete LFs ####
import os
import re

stopwords_pattern = ' of| and| no| others| or| -|,|no '

def ltp(x):
    return '(' + '|'.join(x) + ')'

def create_LF_Based_On_Category_Name(debug=False):
    if os.path.exists("d_Category_Name_LFs.py"):
        os.remove("d_Category_Name_LFs.py")
        
    f = open("d_Category_Name_LFs.py","a+")
    for i in range(len(class_names)):
        functionName = re.sub(r'( )+|-|,','',class_names[i])

        pattern = re.sub(stopwords_pattern , '', class_names[i].lower().strip())
        pattern= re.sub("( )+",",",pattern)
        pattern= re.sub(" $","",pattern)
        
        words = pattern.split(',')
        wordsStr = '['+','.join(['"'+x+'"' for x in words])+']'
        
        pattern = ltp(words)
        if(debug):
            print(pattern)
        f.write("\n")
        f.write(r'''def LF_Category_Name_'''+functionName+'''(c):
    words = '''+wordsStr+'''
    if(len(set(c['complaint_text']).intersection(words))>0):
            return '''+str(i+1)+'''
    return 0''')
        f.write("\n")
    f.close()
    


    

def create_LF_Based_On_Keywords(debug=False):
    if os.path.exists("d_KeyWord_Based_LFs.py"):
        os.remove("d_KeyWord_Based_LFs.py")
    f = open("d_KeyWord_Based_LFs.py","a+")
    for i in range(len(class_names)):
        functionName = re.sub(r'( )+|-|,','',class_names[i])
        
        pattern = re.sub(stopwords_pattern , '', class_names[i].lower().strip())
        pattern= re.sub("( )+",",",pattern)
      
        words = pattern.split(',')
        ##### add keywords #####
        
        words = words+ keywords[class_names[i]]
        ####
        wordsStr = '['+','.join(['"'+x+'"' for x in words])+']'
        
        pattern = ltp(words)
        if(debug):
            print(pattern)
        f.write("\n")
        f.write(r'''def LF_KeyWord_'''+functionName+'''(c):
    words = '''+wordsStr+'''
    if(len(set(c['complaint_text']).intersection(words))>0):
            return '''+str(i+1)+'''
    return 0''')
        f.write("\n")
    f.close()
    
def create_LF_Extracts_Phrases_Regex(debug=False):
    if os.path.exists("d_Regex_Based_Extracts_Phrases_LFs.py"):
        os.remove("d_Regex_Based_Extracts_Phrases_LFs.py")
    f = open("d_Regex_Based_Extracts_Phrases_LFs.py","a+")
    for i in range(len(class_names)):
        if(class_names[i] in extracts_df.index.tolist()):
            functionName = re.sub(r'( )+|-|,','',class_names[i])

            pattern = re.sub(stopwords_pattern , '', class_names[i].lower().strip())
            pattern= re.sub("( )+",",",pattern)

            words = pattern.split(',')
            ##### add keywords #####

            words = words+ keywords[class_names[i]]
            ####
            wordsStr = '['+','.join(['"'+x+'"' for x in words])+']'


            if(debug):
                print(pattern)
            f.write("\n")
            f.write(r'''def LF_Extract_Phrase_Regex_'''+functionName+'''(c):
   
    pattern = \''''+extracts_df.loc[class_names[i]]['complaint words']+'''\'
    if(re.search(pattern,c['complaint_description'],flags=re.I)):
            return '''+str(i+1)+'''
    return 0''')
            f.write("\n")
    f.close()
    
def create_LF_Description_Regex(debug=False):
    if os.path.exists("d_Regex_Based_Description_LFs.py"):
        os.remove("d_Regex_Based_Description_LFs.py")
    f = open("d_Regex_Based_Description_LFs.py","a+")
    for i in range(len(class_names)):
        functionName = re.sub(r'( )+|-|,','',class_names[i])
        
        pattern = re.sub(stopwords_pattern , '', class_names[i].lower().strip())
        pattern= re.sub("( )+",",",pattern)
      
        words = pattern.split(',')
        ##### add keywords #####
        
        words = words+ keywords[class_names[i]]
        ####
        wordsStr = '['+','.join(['"'+x+'"' for x in words])+']'
        
       
        if(debug):
            print(pattern)
        f.write("\n")
        f.write(r'''def LF_Desc_Regex_'''+functionName+'''(c):
    words = '''+wordsStr+'''
    pattern = \''''+''.join(regexPatterns[class_names[i]])+'''\'
    if(re.search(pattern,c['complaint_description'],flags=re.I)):
            return '''+str(i+1)+'''
    return 0''')
        f.write("\n")
    f.close()
    
    
    
def create_LF_Title_Regex(debug=False):
    if os.path.exists("d_Regex_Based_Title_LFs.py"):
        os.remove("d_Regex_Based_Title_LFs.py")
    f = open("d_Regex_Based_Title_LFs.py","a+")
    for i in range(len(class_names)):
        functionName = re.sub(r'( )+|-|,','',class_names[i])
        
        pattern = re.sub(stopwords_pattern , '', class_names[i].lower().strip())
        pattern= re.sub("( )+",",",pattern)
      
        words = pattern.split(',')
        ##### add keywords #####
        
        words = words+ keywords[class_names[i]]
        ####
        wordsStr = '['+','.join(['"'+x+'"' for x in words])+']'
        
        pattern = ltp(words)
        if(debug):
            print(pattern)
        f.write("\n")
        f.write(r'''def LF_Title_Regex_'''+functionName+'''(c):
    words = '''+wordsStr+'''
    pattern = \''''+ ''.join(regexPatterns[class_names[i]]) +'''\'
    if(re.search(pattern,c['complaint_title'],flags=re.I)):
            return '''+str(i+1)+'''
    return 0''')
        f.write("\n")
    f.close()


In [85]:
########### discrete LFs with discrete scores####
import os
import re

stopwords_pattern = ' of| and| no| others| or| -|,|no '

def ltp(x):
    return '(' + '|'.join(x) + ')'

def create_LF_Based_On_Category_Name(debug=False):
    if os.path.exists("d_Category_Name_LFs.py"):
        os.remove("d_Category_Name_LFs.py")
        
    f = open("d_Category_Name_LFs.py","a+")
    for i in range(len(class_names)):
        functionName = re.sub(r'( )+|-|,','',class_names[i])

        pattern = re.sub(stopwords_pattern , '', class_names[i].lower().strip())
        pattern= re.sub("( )+",",",pattern)
        pattern= re.sub(" $","",pattern)
        
        words = pattern.split(',')
        wordsStr = '['+','.join(['"'+x+'"' for x in words])+']'
        
        pattern = ltp(words)
        if(debug):
            print(pattern)
        f.write("\n")
        f.write(r'''def LF_Category_Name_'''+functionName+'''(c):
    words = '''+wordsStr+'''
    if(len(set(c['complaint_text']).intersection(words))>0):
            return ('''+str(i+1)+''',1)
    return (-1,0)''')
        f.write("\n")
    f.close()
    


    

def create_LF_Based_On_Keywords(debug=False):
    if os.path.exists("d_KeyWord_Based_LFs.py"):
        os.remove("d_KeyWord_Based_LFs.py")
    f = open("d_KeyWord_Based_LFs.py","a+")
    for i in range(len(class_names)):
        functionName = re.sub(r'( )+|-|,','',class_names[i])
        
        pattern = re.sub(stopwords_pattern , '', class_names[i].lower().strip())
        pattern= re.sub("( )+",",",pattern)
      
        words = pattern.split(',')
        ##### add keywords #####
        
        words = words+ keywords[class_names[i]]
        ####
        wordsStr = '['+','.join(['"'+x+'"' for x in words])+']'
        
        pattern = ltp(words)
        if(debug):
            print(pattern)
        f.write("\n")
        f.write(r'''def LF_KeyWord_'''+functionName+'''(c):
    words = '''+wordsStr+'''
    if(len(set(c['complaint_text']).intersection(words))>0):
            return ('''+str(i+1)+''',1)
    return (-1,0)''')
        f.write("\n")
    f.close()
    
def create_LF_Extracts_Phrases_Regex(debug=False):
    if os.path.exists("d_Regex_Based_Extracts_Phrases_LFs.py"):
        os.remove("d_Regex_Based_Extracts_Phrases_LFs.py")
    f = open("d_Regex_Based_Extracts_Phrases_LFs.py","a+")
    for i in range(len(class_names)):
        if(class_names[i] in extracts_df.index.tolist()):
            functionName = re.sub(r'( )+|-|,','',class_names[i])

            pattern = re.sub(stopwords_pattern , '', class_names[i].lower().strip())
            pattern= re.sub("( )+",",",pattern)

            words = pattern.split(',')
            ##### add keywords #####

            words = words+ keywords[class_names[i]]
            ####
            wordsStr = '['+','.join(['"'+x+'"' for x in words])+']'


            if(debug):
                print(pattern)
            f.write("\n")
            f.write(r'''def LF_Extract_Phrase_Regex_'''+functionName+'''(c):
   
    pattern = \''''+extracts_df.loc[class_names[i]]['complaint words']+'''\'
    if(re.search(pattern,c['complaint_description'],flags=re.I)):
            return ('''+str(i+1)+''',1)
    return (-1,0)''')
            f.write("\n")
    f.close()
    
def create_LF_Description_Regex(debug=False):
    if os.path.exists("d_Regex_Based_Description_LFs.py"):
        os.remove("d_Regex_Based_Description_LFs.py")
    f = open("d_Regex_Based_Description_LFs.py","a+")
    for i in range(len(class_names)):
        functionName = re.sub(r'( )+|-|,','',class_names[i])
        
        pattern = re.sub(stopwords_pattern , '', class_names[i].lower().strip())
        pattern= re.sub("( )+",",",pattern)
      
        words = pattern.split(',')
        ##### add keywords #####
        
        words = words+ keywords[class_names[i]]
        ####
        wordsStr = '['+','.join(['"'+x+'"' for x in words])+']'
        
       
        if(debug):
            print(pattern)
        f.write("\n")
        f.write(r'''def LF_Desc_Regex_'''+functionName+'''(c):
    words = '''+wordsStr+'''
    pattern = \''''+''.join(regexPatterns[class_names[i]])+'''\'
    if(re.search(pattern,c['complaint_description'],flags=re.I)):
            return ('''+str(i+1)+''',1)
    return (-1,0)''')
        f.write("\n")
    f.close()
    
    
    
def create_LF_Title_Regex(debug=False):
    if os.path.exists("d_Regex_Based_Title_LFs.py"):
        os.remove("d_Regex_Based_Title_LFs.py")
    f = open("d_Regex_Based_Title_LFs.py","a+")
    for i in range(len(class_names)):
        functionName = re.sub(r'( )+|-|,','',class_names[i])
        
        pattern = re.sub(stopwords_pattern , '', class_names[i].lower().strip())
        pattern= re.sub("( )+",",",pattern)
      
        words = pattern.split(',')
        ##### add keywords #####
        
        words = words+ keywords[class_names[i]]
        ####
        wordsStr = '['+','.join(['"'+x+'"' for x in words])+']'
        
        pattern = ltp(words)
        if(debug):
            print(pattern)
        f.write("\n")
        f.write(r'''def LF_Title_Regex_'''+functionName+'''(c):
    words = '''+wordsStr+'''
    pattern = \''''+ ''.join(regexPatterns[class_names[i]]) +'''\'
    if(re.search(pattern,c['complaint_title'],flags=re.I)):
            return ('''+str(i+1)+''',1)
    return (-1,0)''')
        f.write("\n")
    f.close()


In [86]:
LF_Names = []
LF_output_map = dict()
LFs_l = []
    
create_LF_Title_Regex()
create_LF_Description_Regex()
create_LF_Based_On_Keywords()
create_LF_Extracts_Phrases_Regex()
# create_LF_Based_On_Category_Name()
# create_LF_Based_On_Embeddings()
# create_LF_Based_On_TFIDF()
# create_LF_Based_On_Embeddings_Title()
# create_LF_Based_On_Embeddings_Description()



        
# for i in range(len(class_names)):
#         functionName = re.sub(r'( )+|-|,','',class_names[i])
#         LF_Names.append('LF_Category_Name_'+functionName)
#         LF_output_map['LF_Category_Name_'+functionName]=i
        
# for i in range(len(class_names)):
#         functionName = re.sub(r'( )+|-|,','',class_names[i])
#         LF_Names.append('LF_Embedding_'+functionName)
#         LF_output_map['LF_Embedding_'+functionName]=i
        

        
# for i in range(len(class_names)):
#         functionName = re.sub(r'( )+|-|,','',class_names[i])
#         LF_Names.append('LF_TFIDF_'+functionName)
#         LF_output_map['LF_TFIDF_'+functionName]=i

for i in range(len(class_names)):
        functionName = re.sub(r'( )+|-|,','',class_names[i])
        LF_Names.append('LF_KeyWord_'+functionName)
        LF_output_map['LF_KeyWord_'+functionName]=i
        LFs_l.append(i)

for i in range(len(class_names)):
        functionName = re.sub(r'( )+|-|,','',class_names[i])
        LF_Names.append('LF_Title_Regex_'+functionName)
        LF_output_map['LF_Title_Regex_'+functionName]=i
        LFs_l.append(i)

for i in range(len(class_names)):
        functionName = re.sub(r'( )+|-|,','',class_names[i])
        LF_Names.append('LF_Desc_Regex_'+functionName)
        LF_output_map['LF_Desc_Regex_'+functionName]=i
        LFs_l.append(i)

for i in range(len(class_names)):
    if(class_names[i] in extracts_df.index.tolist()):
        functionName = re.sub(r'( )+|-|,','',class_names[i])
        LF_Names.append('LF_Extract_Phrase_Regex_'+functionName)
        LF_output_map['LF_Extract_Phrase_Regex_'+functionName]=i
        LFs_l.append(i)
        
# for i in range(len(class_names)):
#         functionName = re.sub(r'( )+|-|,','',class_names[i])
#         LF_Names.append('LF_Title_Embedding_'+functionName)
#         LF_output_map['LF_Title_Embedding_'+functionName]=i
        
# for i in range(len(class_names)):
#         functionName = re.sub(r'( )+|-|,','',class_names[i])
#         LF_Names.append('LF_Description_Embedding_'+functionName)
#         LF_output_map['LF_Description_Embedding_'+functionName]=i
 

        

print('['+','.join(LF_Names)+']')
LFs_l = np.array(LFs_l)


[LF_KeyWord_AirPollution,LF_KeyWord_AutorickshawsandTaxis,LF_KeyWord_BMTCDriverorConductor,LF_KeyWord_BMTCNeednewBusRoute,LF_KeyWord_BMTCOthers,LF_KeyWord_BadRoads,LF_KeyWord_BrokenStormWaterDrains,LF_KeyWord_Cattle,LF_KeyWord_ClearingofBlockageofUnderGroundDrainagePipelinesandReplacementofDamagedorMissingManholeCover,LF_KeyWord_DesiltingLakes,LF_KeyWord_Diseases,LF_KeyWord_Electricity,LF_KeyWord_FloodingofRoadsandFootpaths,LF_KeyWord_Footpaths,LF_KeyWord_Garbage,LF_KeyWord_GovernmentLandEncroachment,LF_KeyWord_HawkersandVendors,LF_KeyWord_Hoardings,LF_KeyWord_IllegalpostersandHoardings,LF_KeyWord_LakesOthers,LF_KeyWord_MaintenanceofRoadsandFootpathsOthers,LF_KeyWord_Manholes,LF_KeyWord_Mosquitos,LF_KeyWord_NeedNewStreetlights,LF_KeyWord_NeedNewToilets,LF_KeyWord_NewBusShelters,LF_KeyWord_NoSewageDrains,LF_KeyWord_NoisePollution,LF_KeyWord_Others,LF_KeyWord_OverflowofStormWaterDrains,LF_KeyWord_ParkingViolations,LF_KeyWord_Parksandplaygrounds,LF_KeyWord_Potholes,LF_KeyWord_PublicNuisan

In [88]:
%load d_KeyWord_Based_LFs.py

In [90]:
%load d_Regex_Based_Title_LFs.py

In [92]:
%load d_Regex_Based_Description_LFs.py

In [94]:
%load d_Regex_Based_Extracts_Phrases_LFs.py

In [95]:
LFs = [LF_KeyWord_AirPollution,LF_KeyWord_AutorickshawsandTaxis,LF_KeyWord_BMTCDriverorConductor,LF_KeyWord_BMTCNeednewBusRoute,LF_KeyWord_BMTCOthers,LF_KeyWord_BadRoads,LF_KeyWord_BrokenStormWaterDrains,LF_KeyWord_Cattle,LF_KeyWord_ClearingofBlockageofUnderGroundDrainagePipelinesandReplacementofDamagedorMissingManholeCover,LF_KeyWord_DesiltingLakes,LF_KeyWord_Diseases,LF_KeyWord_Electricity,LF_KeyWord_FloodingofRoadsandFootpaths,LF_KeyWord_Footpaths,LF_KeyWord_Garbage,LF_KeyWord_GovernmentLandEncroachment,LF_KeyWord_HawkersandVendors,LF_KeyWord_Hoardings,LF_KeyWord_IllegalpostersandHoardings,LF_KeyWord_LakesOthers,LF_KeyWord_MaintenanceofRoadsandFootpathsOthers,LF_KeyWord_Manholes,LF_KeyWord_Mosquitos,LF_KeyWord_NeedNewStreetlights,LF_KeyWord_NeedNewToilets,LF_KeyWord_NewBusShelters,LF_KeyWord_NoSewageDrains,LF_KeyWord_NoisePollution,LF_KeyWord_Others,LF_KeyWord_OverflowofStormWaterDrains,LF_KeyWord_ParkingViolations,LF_KeyWord_Parksandplaygrounds,LF_KeyWord_Potholes,LF_KeyWord_PublicNuisance,LF_KeyWord_Repairofstreetlights,LF_KeyWord_SewageandStormWaterDrainsOthers,LF_KeyWord_StrayDogs,LF_KeyWord_Traffic,LF_KeyWord_TreesParksandPlaygroundsOthers,LF_KeyWord_UnauthorizedConstruction,LF_KeyWord_WaterLeakage,LF_KeyWord_WaterSupply,LF_Title_Regex_AirPollution,LF_Title_Regex_AutorickshawsandTaxis,LF_Title_Regex_BMTCDriverorConductor,LF_Title_Regex_BMTCNeednewBusRoute,LF_Title_Regex_BMTCOthers,LF_Title_Regex_BadRoads,LF_Title_Regex_BrokenStormWaterDrains,LF_Title_Regex_Cattle,LF_Title_Regex_ClearingofBlockageofUnderGroundDrainagePipelinesandReplacementofDamagedorMissingManholeCover,LF_Title_Regex_DesiltingLakes,LF_Title_Regex_Diseases,LF_Title_Regex_Electricity,LF_Title_Regex_FloodingofRoadsandFootpaths,LF_Title_Regex_Footpaths,LF_Title_Regex_Garbage,LF_Title_Regex_GovernmentLandEncroachment,LF_Title_Regex_HawkersandVendors,LF_Title_Regex_Hoardings,LF_Title_Regex_IllegalpostersandHoardings,LF_Title_Regex_LakesOthers,LF_Title_Regex_MaintenanceofRoadsandFootpathsOthers,LF_Title_Regex_Manholes,LF_Title_Regex_Mosquitos,LF_Title_Regex_NeedNewStreetlights,LF_Title_Regex_NeedNewToilets,LF_Title_Regex_NewBusShelters,LF_Title_Regex_NoSewageDrains,LF_Title_Regex_NoisePollution,LF_Title_Regex_Others,LF_Title_Regex_OverflowofStormWaterDrains,LF_Title_Regex_ParkingViolations,LF_Title_Regex_Parksandplaygrounds,LF_Title_Regex_Potholes,LF_Title_Regex_PublicNuisance,LF_Title_Regex_Repairofstreetlights,LF_Title_Regex_SewageandStormWaterDrainsOthers,LF_Title_Regex_StrayDogs,LF_Title_Regex_Traffic,LF_Title_Regex_TreesParksandPlaygroundsOthers,LF_Title_Regex_UnauthorizedConstruction,LF_Title_Regex_WaterLeakage,LF_Title_Regex_WaterSupply,LF_Desc_Regex_AirPollution,LF_Desc_Regex_AutorickshawsandTaxis,LF_Desc_Regex_BMTCDriverorConductor,LF_Desc_Regex_BMTCNeednewBusRoute,LF_Desc_Regex_BMTCOthers,LF_Desc_Regex_BadRoads,LF_Desc_Regex_BrokenStormWaterDrains,LF_Desc_Regex_Cattle,LF_Desc_Regex_ClearingofBlockageofUnderGroundDrainagePipelinesandReplacementofDamagedorMissingManholeCover,LF_Desc_Regex_DesiltingLakes,LF_Desc_Regex_Diseases,LF_Desc_Regex_Electricity,LF_Desc_Regex_FloodingofRoadsandFootpaths,LF_Desc_Regex_Footpaths,LF_Desc_Regex_Garbage,LF_Desc_Regex_GovernmentLandEncroachment,LF_Desc_Regex_HawkersandVendors,LF_Desc_Regex_Hoardings,LF_Desc_Regex_IllegalpostersandHoardings,LF_Desc_Regex_LakesOthers,LF_Desc_Regex_MaintenanceofRoadsandFootpathsOthers,LF_Desc_Regex_Manholes,LF_Desc_Regex_Mosquitos,LF_Desc_Regex_NeedNewStreetlights,LF_Desc_Regex_NeedNewToilets,LF_Desc_Regex_NewBusShelters,LF_Desc_Regex_NoSewageDrains,LF_Desc_Regex_NoisePollution,LF_Desc_Regex_Others,LF_Desc_Regex_OverflowofStormWaterDrains,LF_Desc_Regex_ParkingViolations,LF_Desc_Regex_Parksandplaygrounds,LF_Desc_Regex_Potholes,LF_Desc_Regex_PublicNuisance,LF_Desc_Regex_Repairofstreetlights,LF_Desc_Regex_SewageandStormWaterDrainsOthers,LF_Desc_Regex_StrayDogs,LF_Desc_Regex_Traffic,LF_Desc_Regex_TreesParksandPlaygroundsOthers,LF_Desc_Regex_UnauthorizedConstruction,LF_Desc_Regex_WaterLeakage,LF_Desc_Regex_WaterSupply,LF_Extract_Phrase_Regex_AirPollution,LF_Extract_Phrase_Regex_BMTCDriverorConductor,LF_Extract_Phrase_Regex_BMTCNeednewBusRoute,LF_Extract_Phrase_Regex_BMTCOthers,LF_Extract_Phrase_Regex_BadRoads,LF_Extract_Phrase_Regex_BrokenStormWaterDrains,LF_Extract_Phrase_Regex_ClearingofBlockageofUnderGroundDrainagePipelinesandReplacementofDamagedorMissingManholeCover,LF_Extract_Phrase_Regex_DesiltingLakes,LF_Extract_Phrase_Regex_Diseases,LF_Extract_Phrase_Regex_Electricity,LF_Extract_Phrase_Regex_FloodingofRoadsandFootpaths,LF_Extract_Phrase_Regex_Footpaths,LF_Extract_Phrase_Regex_Garbage,LF_Extract_Phrase_Regex_HawkersandVendors,LF_Extract_Phrase_Regex_IllegalpostersandHoardings,LF_Extract_Phrase_Regex_LakesOthers,LF_Extract_Phrase_Regex_MaintenanceofRoadsandFootpathsOthers,LF_Extract_Phrase_Regex_Manholes,LF_Extract_Phrase_Regex_Mosquitos,LF_Extract_Phrase_Regex_NeedNewStreetlights,LF_Extract_Phrase_Regex_NewBusShelters,LF_Extract_Phrase_Regex_NoSewageDrains,LF_Extract_Phrase_Regex_NoisePollution,LF_Extract_Phrase_Regex_Others,LF_Extract_Phrase_Regex_OverflowofStormWaterDrains,LF_Extract_Phrase_Regex_ParkingViolations,LF_Extract_Phrase_Regex_Parksandplaygrounds,LF_Extract_Phrase_Regex_Potholes,LF_Extract_Phrase_Regex_PublicNuisance,LF_Extract_Phrase_Regex_Repairofstreetlights,LF_Extract_Phrase_Regex_SewageandStormWaterDrainsOthers,LF_Extract_Phrase_Regex_StrayDogs,LF_Extract_Phrase_Regex_Traffic,LF_Extract_Phrase_Regex_TreesParksandPlaygroundsOthers,LF_Extract_Phrase_Regex_WaterLeakage]

In [96]:
pkl.dump(LF_Names,open("LF_Names.p","wb"))
pkl.dump(LF_output_map,open("LF_output_map.p","wb"))

print(len(LF_Names))
print(len(LF_output_map))
print(LFs_l.shape)

161
161
(161,)


In [ ]:
# returns: NoOf samples by NoOf LFs list of lists
def get_L_S_Tensor(df,msg): 
    L = []
    print('labelling ',msg,' data')
    for i in range(len(df.index)):
        Li=[]
        for LF in LFs:
#             print(i,LF.__name__)            
            l = LF(df.iloc[i])
            Li.append(l)
#             S.append((s+1)/2)  #to scale scores in [0,1] 
        L.append(Li)
        if(i%500==0 and i!=0):
            print(str(i)+'data points labelled in',(time.time() - start_time)/60,'mins')
        
    return L

import time
import datetime
start_time = time.time()

lt = time.localtime()

print("started at: {}-{}-{}, {}:{}:{}".format(lt.tm_mday,lt.tm_mon,lt.tm_year,lt.tm_hour,lt.tm_min,lt.tm_sec))


dev_L_S = get_L_S_Tensor(test_df,'discrete test')
np.save("dev_L_S_discrete",np.array(dev_L_S))

train_L_S = get_L_S_Tensor(train_df,'discrete train')
np.save("train_L_S_discrete",np.array(train_L_S))


print(str(datetime.timedelta(seconds=time.time() - start_time)))


In [98]:
''' output:

    [[[L_x1],[S_x1]],
     [[L_x2],[S_x2]],
     ......
     ......
    ]

'''
def get_L_S_Tensor(df,msg):     
    L_S = []
    print('labelling ',msg,' data')
    for i in range(len(df.index)):
        L_S_ci=[]
        L=[]
        S=[]
        P_ik = []
        for LF in LFs:
#             print(i,LF.__name__)            
            l,s = LF(df.iloc[i])
            L.append(l)
            S.append((s+1)/2)  #to scale scores in [0,1] 
        L_S_ci.append(L)
        L_S_ci.append(S)
        L_S.append(L_S_ci) 
        if(i%500==0 and i!=0):
            print(str(i)+'data points labelled in',(time.time() - start_time)/60,'mins')
        
    return L_S


import time
import datetime
start_time = time.time()

lt = time.localtime()

print("started at: {}-{}-{}, {}:{}:{}".format(lt.tm_mday,lt.tm_mon,lt.tm_year,lt.tm_hour,lt.tm_min,lt.tm_sec))


dev_L_S = get_L_S_Tensor(test_df,'discrete test')
np.save("dev_L_S_discrete",np.array(dev_L_S))

train_L_S = get_L_S_Tensor(train_df,'discrete train')
np.save("train_L_S_discrete",np.array(train_L_S))


print(str(datetime.timedelta(seconds=time.time() - start_time)))



started at: 19-4-2018, 16:4:51
labelling  discrete test  data
500data points labelled in 0.16915095647176107 mins
1000data points labelled in 0.32821693023045856 mins
1500data points labelled in 0.4837478041648865 mins
2000data points labelled in 0.6394636193911235 mins
2500data points labelled in 0.7950547575950623 mins
3000data points labelled in 0.9511594772338867 mins
3500data points labelled in 1.1070024927457174 mins
4000data points labelled in 1.262646782398224 mins
4500data points labelled in 1.4175927797953287 mins
5000data points labelled in 1.5732842048009237 mins
5500data points labelled in 1.7309389114379883 mins
6000data points labelled in 1.8874680956204732 mins
6500data points labelled in 2.044269684950511 mins
labelling  discrete train  data
500data points labelled in 2.2963799079259237 mins
1000data points labelled in 2.4928781270980833 mins
1500data points labelled in 2.6909250100453694 mins
2000data points labelled in 2.8881229043006895 mins
2500data points labelled

In [49]:
import scipy.sparse as sp
import _pickle as pkl
L_train = pkl.load(open("train_L_S_discrete.p","rb"))
L_train = sp.csr_matrix(L_train)

In [1]:
import os
import numpy as np

from snorkel import SnorkelSession
session = SnorkelSession()

In [2]:
from snorkel.learning import GenerativeModel

gen_model = GenerativeModel()

Instructions for updating:
Use the retry module or similar alternatives.


In [50]:
# Note: We pass cardinality explicitly here to be safe
# Can usually be inferred, except we have no labels with value=3

start_time = time.time()

lt = time.localtime()

print("started at: {}-{}-{}, {}:{}:{}".format(lt.tm_mday,lt.tm_mon,lt.tm_year,lt.tm_hour,lt.tm_min,lt.tm_sec))

gen_model.train(L_train, cardinality=42)
# gen_model.train(L_train, epochs=100, decay=0.95, step_size=0.1 / L_train.shape[0], reg_param=1e-6)

print(str(datetime.timedelta(seconds=time.time() - start_time)))

started at: 13-4-2018, 23:2:25
1:32:12.027037


In [79]:
train_marginals = gen_model.marginals(L_train)
train_marginals = np.array(train_marginals)
train_GenLabels = np.argmax(train_marginals,axis=1)
print(train_GenLabels.shape)
train_GenLabels = [class_names[x] for x in train_GenLabels]
train_df["predicted_label"]=train_GenLabels
train_df.to_csv("ICMCwithSnorkelPredictedLabels.csv")

(37595,)


In [68]:
L_test = pkl.load(open("test_L_S_discrete.p","rb"))
L_test = sp.csr_matrix(L_test)

In [72]:
import numpy as np
dev_marginals = gen_model.marginals(L_test)
dev_marginals = np.array(dev_marginals)
print(dev_marginals.shape)

GenLabels = np.argmax(dev_marginals,axis=1)
print(GenLabels.shape)


(6667, 42)
(6667,)


In [120]:
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from collections import defaultdict
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

def report2dict(cr):
    # Parse rows
    tmp = list()
    for row in cr.split("\n"):
        parsed_row = [x for x in row.split("  ") if len(x) > 0]
        if len(parsed_row) > 0:
            tmp.append(parsed_row)
    
    # Store in dictionary
    measures = tmp[0]

    D_class_data = defaultdict(dict)
    for row in tmp[1:]:
        class_label = row[0]
        for j, m in enumerate(measures):
            D_class_data[class_label][m.strip()] = float(row[j + 1].strip())
    return pd.DataFrame(D_class_data).T

def predictAndPrint(true_labels,pl):
    print("acc",accuracy_score(true_labels,pl))
#     print(precision_recall_fscore_support(true_labels,pl,average='macro'))
#     draw2DArray(confusion_matrix(true_labels,pl))
    return report2dict(classification_report(true_labels, pl, target_names=class_names))
    

In [33]:
#load true test labels
import pandas as pd

test_df=pd.read_csv('./complaints_test_data_clean.csv',usecols=["category_name","complaint_title","complaint_description",],na_filter=False)

colsize = len(test_df['category_name'])

test_df['category_name'] = test_df["category_name"].astype('category')
test_df['true_label'] = test_df['category_name'].cat.codes

true_labels = test_df['true_label'].tolist()
gold_labels_dev = true_labels
test_df

,category_name,complaint_title,complaint_description,true_label
0,Maintenance of Roads and Footpaths - Others,Mud road yet to be tarred,We had a nice tar road in Kannamangala connect...,20
1,Public Nuisance,Policemen creating discomfort,Most of the time some 2 policemen from Bangalo...,33
2,Hoardings,Illegal hoardings on bellary road,Illegal hoardings on bellary road on servies r...,17
3,Repair of streetlights,Street Light Unavailable for several Years,"In our layout, Margosa Avenue opposite next to...",34
4,Traffic,Traffic issues near KG Halli entrance,Traffic issues near KG Halli entrance,37
5,Bad Roads,"No Tar road near Vaswani Centropolis, Shanti N...",There is no Tar road beside Vaswani Centropolis.,5
6,Garbage,Daily garbage left on road,Daily garbage collected on road and uncleared,14
7,Potholes,"Fill pot holes on CMH Road,Indiranagar,Hoysala...",There are lot of pot holes on Chinmayya missio...,32
8,Sewage and Storm Water Drains - Others,Irresponsible &amp; Apathetic : &lt; 100\' fee...,Drain construction work in front of Ittina Sou...,35
9,Garbage,Dumping of garbage,Garbage is dumped near our compound which has ...,14


In [60]:
Results = predictAndPrint(true_labels,GenLabels)
Results

acc 0.568021598920054


,f1-score,precision,recall,support
BMTC - Driver or Conductor,0.42,0.67,0.31,26.0
Cattle,0.50,0.57,0.44,9.0
Diseases,0.10,0.25,0.06,49.0
Government Land Encroachment,0.11,0.09,0.12,8.0
Lakes - Others,0.24,0.14,0.73,11.0
Manholes,0.27,0.19,0.46,13.0
New Bus Shelters,0.30,0.53,0.21,121.0
No Sewage Drains,0.26,0.19,0.41,100.0
Others,0.00,0.00,0.00,131.0
Overflow of Storm Water Drains,0.00,0.00,0.00,41.0


In [75]:
#majority
import math
L_test = pkl.load(open("test_L_S_discrete.p","rb"))
L_test = pd.DataFrame(L_test)
L_test = L_test.replace(0, np.NaN)
predicted_labels=L_test.mode(axis=1)[0].tolist()
# print(predicted_labels)
predicted_labels = [ int(x)-1 if not math.isnan(x) else -1 for x in predicted_labels ]
MajLabels = predicted_labels
Results = predictAndPrint(true_labels,predicted_labels)
Results

acc 0.5696715164241788


/home/vinay/snorkelEnv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 43, does not match size of target_names, 42
  .format(len(labels), len(target_names))
/home/vinay/snorkelEnv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/vinay/snorkelEnv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


,f1-score,precision,recall,support
BMTC - Driver or Conductor,0.41,0.27,0.83,18.0
Cattle,0.07,0.12,0.05,20.0
Diseases,0.20,0.12,0.70,10.0
Government Land Encroachment,0.88,0.88,0.88,1319.0
Lakes - Others,0.21,0.13,0.64,14.0
Manholes,0.00,0.00,0.00,27.0
New Bus Shelters,0.60,0.59,0.62,21.0
No Sewage Drains,0.35,0.93,0.21,121.0
Others,0.54,0.57,0.52,31.0
Overflow of Storm Water Drains,0.00,0.00,0.00,131.0


In [35]:
NoOfLFs= len(LFs)
NoOfClasses = len(class_names)

In [100]:
import numpy as np
dev_L_S = np.load("dev_L_S_discrete.npy")
train_L_S = np.load("train_L_S_discrete.npy")
print(dev_L_S.shape,train_L_S.shape)

(6667, 2, 161) (37595, 2, 161)


In [135]:
## multi class
from __future__ import absolute_import, division, print_function

import tensorflow as tf

BATCH_SIZE = 1
tf.reset_default_graph()


with tf.Graph().as_default():
    
    train_dataset = tf.data.Dataset.from_tensor_slices(train_L_S).batch(BATCH_SIZE)
    dev_dataset = tf.data.Dataset.from_tensor_slices(dev_L_S).batch(dev_L_S.shape[0])

    labels = tf.convert_to_tensor(gold_labels_dev)
    iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
    next_element = iterator.get_next()

    train_init_op = iterator.make_initializer(train_dataset)
    dev_init_op = iterator.make_initializer(dev_dataset)
   
    next_element = iterator.get_next()
    print("next_element",next_element)
    

    alphas = tf.get_variable('alphas', [NoOfLFs,NoOfClasses],\
                             initializer=tf.truncated_normal_initializer(0.2,0.1,12),\
                             dtype=tf.float64)

    thetas = tf.get_variable('thetas', [NoOfLFs,NoOfClasses],\
                             initializer=tf.truncated_normal_initializer(1,0.2,12),\
                             dtype=tf.float64)
    print(alphas.graph)
    print(thetas.graph)
    
    print("alphas",alphas)
    print("thetas",thetas)    
    l,s =  tf.unstack(next_element,axis=1)
    print("s",s)
    print("l",l)
    
    lc = tf.one_hot(tf.cast(l,tf.int32),depth=NoOfClasses,dtype=tf.float64)
    print("lc",lc)
    sc = tf.multiply(lc, tf.expand_dims(s,-1))
    
    print("sc",sc)


    sc_ = tf.map_fn(lambda x : tf.maximum(tf.subtract(x,alphas), 0), sc)
    print("s_",sc)
            
    LF_c = tf.cast(tf.one_hot(LFs_l,depth=NoOfClasses,on_value=1.0,off_value=-1.0),dtype=tf.float64)*lc
    print("LF_c",LF_c)

    
    pout = LF_c*lc # LF_c * sc_
#     pout = tf.map_fn(lambda x: lc*x,np.arange(NoOfClasses,dtype=np.float64))
#     print("nls",nls_)
    print("thetas",thetas)
    print("phi_out",pout)

#     t_pout = tf.map_fn(lambda x: tf.matmul(x,thetas),pout)
    t_pout = pout * thetas
 
    print("t_pout",t_pout)
    P = tf.reduce_sum(t_pout,1)
    loss = tf.reduce_sum(tf.reduce_logsumexp(P,1))
    print("P",P)
    print("loss",loss)

    t_lc =  lc*thetas*LF_c # sc_*thetas*LF_c
    print("t_l",t_l)
    
#     zy = tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t_l*y),axis=1),np.arange([-1,1],dtype=np.float64))
    
    zy = tf.reduce_prod(1 + tf.exp(t_lc),axis=1)
    logz = tf.log(tf.reduce_sum(zy,axis=1))
    print("zy",zy)
    print("logz",logz)
    
#     lsp = tf.reduce_logsumexp(t_pout)
#     print("lsp",lsp)

    normloss = tf.negative(loss - logz)  

    marginals = tf.nn.softmax(P,axis=1)
      
    print("marginals",marginals)
    predict = tf.argmax(marginals,axis=1)
    print("predict",predict)
    
#     pre = tf.metrics.precision(labels,predict)
#     rec = tf.metrics.recall(labels,predict)
#     print("loss",loss)
#     print("nls_",nls_)

#     global_step = tf.Variable(0, trainable=False,dtype=tf.float64)
#     starter_learning_rate = 1
#     learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
#                                            100000, 0.96, staircase=True)


#     train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=global_step) 


#     train_step = tf.train.AdamOptimizer(0.1).minimize(normloss) 
    train_step = tf.train.AdagradOptimizer(0.1).minimize(normloss) 
#     train_step = tf.train.GradientDescentOptimizer(0.1).minimize(normloss) 

    init_g = tf.global_variables_initializer()
    init_l = tf.local_variables_initializer()
    with tf.Session() as sess:
        sess.run(init_g)
        sess.run(init_l)
        
        # Initialize an iterator over the training dataset.
        for it in range(50):
            sess.run(train_init_op)
            tl = 0
            try:
                while True:
                    _,l = sess.run([train_step,normloss])
                    tl = tl + l
            except tf.errors.OutOfRangeError:
                pass
            print(it,"loss",tl)
        # Initialize an iterator over the validation dataset.
        sess.run(dev_init_op)
        a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
#         print(a)
        print(t)
          
        unique, counts = np.unique(pl, return_counts=True)
        print(dict(zip(unique, counts)))
        print("acc",accuracy_score(true_labels,pl))
#             print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl)))
        print(precision_recall_fscore_support(np.array(true_labels),np.array(pl),average="macro"))
        Results = predictAndPrint(true_labels,pl)
        Results.to_csv("results.csv")
Results
           

next_element Tensor("IteratorGetNext_1:0", shape=(?, 2, 161), dtype=float64)
alphas <tf.Variable 'alphas:0' shape=(161, 42) dtype=float64_ref>
thetas <tf.Variable 'thetas:0' shape=(161, 42) dtype=float64_ref>
s Tensor("unstack:1", shape=(?, 161), dtype=float64)
l Tensor("unstack:0", shape=(?, 161), dtype=float64)
lc Tensor("one_hot:0", shape=(?, 161, 42), dtype=float64)
sc Tensor("Mul:0", shape=(?, 161, 42), dtype=float64)
s_ Tensor("Mul:0", shape=(?, 161, 42), dtype=float64)
LF_c Tensor("mul:0", shape=(?, 161, 42), dtype=float64)
thetas <tf.Variable 'thetas:0' shape=(161, 42) dtype=float64_ref>
phi_out Tensor("mul_1:0", shape=(?, 161, 42), dtype=float64)
t_pout Tensor("mul_2:0", shape=(?, 161, 42), dtype=float64)
P Tensor("Sum:0", shape=(?, 42), dtype=float64)
loss Tensor("Sum_1:0", shape=(), dtype=float64)
t_l Tensor("mul_4:0", shape=(?, 161, 42), dtype=float64)
zy Tensor("Prod:0", shape=(?, 42), dtype=float64)
logz Tensor("Log:0", shape=(?,), dtype=float64)
marginals Tensor("Softmax

/home/vinay/snorkelEnv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,f1-score,precision,recall,support
BMTC - Driver or Conductor,0.00,0.00,0.00,26.0
Cattle,0.00,0.00,0.00,9.0
Diseases,0.00,0.00,0.00,49.0
Government Land Encroachment,0.00,0.00,0.00,8.0
Lakes - Others,0.00,0.00,0.00,11.0
Manholes,0.00,0.00,0.00,13.0
New Bus Shelters,0.00,0.00,0.00,121.0
No Sewage Drains,0.00,0.00,0.00,100.0
Others,0.01,0.02,0.01,131.0
Overflow of Storm Water Drains,0.00,0.00,0.00,41.0
